In [1]:
import json
import os
import bz2
import io
from bz2 import BZ2File
import pandas as pd
import numpy as np
import re
from utils import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

In [2]:
files_dir = "../data/impresso-data"
archives = os.listdir(files_dir)
date = "1969"

* Extract interesting archives

In [3]:
regex = '(.*196[8-9].*)|(.*197[0-1].*)'
gdl_ = extract_particular_files(archives, regex, 'GDL')      
jdg_ = extract_particular_files(archives, regex, 'JDG') 

* Extract content  and preprocessing

In [4]:
 df_all, df_gdl, df_jdg = run(gdl_, jdg_, date, files_dir)

GDL-1968.jsonl.bz2 : 51434 articles à extraire
GDL-1969.jsonl.bz2 : 50467 articles à extraire
GDL-1970.jsonl.bz2 : 58656 articles à extraire
GDL-1971.jsonl.bz2 : 54695 articles à extraire
JDG-1968.jsonl.bz2 : 45200 articles à extraire
JDG-1969.jsonl.bz2 : 47098 articles à extraire
JDG-1970.jsonl.bz2 : 45544 articles à extraire
JDG-1971.jsonl.bz2 : 44609 articles à extraire


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,mag,date,page,ppage,text,length
233,GDL,1968-07-27,1,NaN,L'échec de Moscou BULLETIN Au-delà des rumeurs contradictoires qui sont le lot de toute guerre d...,4135
234,GDL,1968-07-27,2,NaN,\ iomus >\\ I////LOUIJ //|\ \ v . Magnifiques circuits en cars pullman 3 Jours : 28-30 juillet C...,873
235,GDL,1968-07-27,3,NaN,"MÉCANISME INDISPENSABLE AU DÉVELOPPEMENT INDUSTRIEL LA BOURSE DANS L'ECONOMIE SUISSE.-V, par Gér...",7341
236,GDL,1968-07-27,4,NaN,"M. Pellegrini acquitté TESSIN « H Lavoro », l'hebdomadaire des syndicats chrétiens-sociaux du Te...",1494
237,GDL,1968-07-27,5,NaN,Pour une politique agricole cohérente LA CHAMBRE VAUDOISE D'AGRICULTURE VIENT DE PUBLIER UN DEUX...,5349


Length df: 131      430
71       424
200      419
201      409
68       378
        ... 
16745      1
30178      1
19048      1
29295      1
10093      1
Name: length, Length: 12524, dtype: int64


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,mag,date,page,ppage,text,length
233,GDL,1968-07-27,1,NaN,L'échec de Moscou BULLETIN Au-delà des rumeurs contradictoires qui sont le lot de toute guerre d...,4135
234,GDL,1968-07-27,2,NaN,\ iomus >\\ I////LOUIJ //|\ \ v . Magnifiques circuits en cars pullman 3 Jours : 28-30 juillet C...,873
235,GDL,1968-07-27,3,NaN,"MÉCANISME INDISPENSABLE AU DÉVELOPPEMENT INDUSTRIEL LA BOURSE DANS L'ECONOMIE SUISSE.-V, par Gér...",7341
236,GDL,1968-07-27,4,NaN,"M. Pellegrini acquitté TESSIN « H Lavoro », l'hebdomadaire des syndicats chrétiens-sociaux du Te...",1494
237,GDL,1968-07-27,5,NaN,Pour une politique agricole cohérente LA CHAMBRE VAUDOISE D'AGRICULTURE VIENT DE PUBLIER UN DEUX...,5349


Length df: 71       358
131      320
68       310
200      255
107      244
        ... 
11361      1
11739      1
7828       1
9690       1
5993       1
Name: length, Length: 9468, dtype: int64


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,mag,date,page,ppage,text,length
0,JDG,1968-07-20,1,NaN,Qui est le vrai Couve de Murville ? (De notre correspondant de Paris) • Le nouveau gouvernement ...,4516
1,JDG,1968-07-20,2,NaN,"Vente 'en faveur du « Teatro popolare viaggiante » Chiffres faussés ? Lucerne, 19.-(ATS). La gra...",1628
2,JDG,1968-07-20,3,NaN,"Nouveau Cabinet irakien •• Bagdad, 19.-(AFP) Le nouveau gouvernement irakien, formé jeudi soir p...",1485
3,JDG,1968-07-20,4,NaN,"Sympathie toujours, maIs aussI se 've' .,, ; te'menacante • • , If A en croire certains passages...",14899
4,JDG,1968-07-20,5,NaN,"No 168-20-21 juillet. 1968 'JOURNAL DI ! : GE ~ EVE 5 E ,: ~ L ..': Gêi, v ~': rzn 7 ~ =: mlK. ,...",15433


Length df: 201      185
215      178
203      173
214      172
138      170
        ... 
9015       1
11062      1
7792       1
12981      1
5949       1
Name: length, Length: 11135, dtype: int64


* Keywords filtering

In [5]:
#keywords_1962 =['nucléaire', 'Cuba', 'guerre', 'tension','risque', 'bombe', 'guerre nucléaire']
#keywords_1984 = ['nucléaire','initiatives', 'initiative', 'centrale', 'centrales', 'atomique', 'atomiques', 'energie', 'votation']
keywords_1969 = ['nucléaire',
                    'centrale nucléaire',
                    'lucens',
                    'uranium',
                    'fission',
                    'atome',
                    'Beznau',
                    'Gösgen',
                    'Leibstadt',
                    'Mühleberg',
                    'réacteurs', 
                    'déchets nucléaires',
                    'accident nucléaire']
cleaned_files_dir = "../data/filtered/"
unwanted = ['tableau hebdomadaire']#, '(\/(\s\d*)*\s)*']
archives = os.listdir(files_dir)

In [6]:
#df_all = load_corpus(cleaned_files_dir + "cleaned_all" + date +".json.bz2")
#df_gdl = load_corpus(cleaned_files_dir + "cleaned_gdl" + date +".json.bz2")
#df_jdg = load_corpus(cleaned_files_dir + "cleaned_jdg" + date +".json.bz2")



In [7]:
df_all_ = keywords_filtering(df_all, keywords_1969)
df_all_ = unwanted_filtering(df_all_, unwanted)


===KEYWORDS===


'After keyword filtered: '

(1569, 20)

=====UNWANTED====


'After keyword filtered: '

(1569, 20)

In [8]:
df_gdl_ = keywords_filtering(df_gdl, keywords_1969)
df_gdl_ = unwanted_filtering(df_gdl_, unwanted)

===KEYWORDS===


'After keyword filtered: '

(706, 20)

=====UNWANTED====


'After keyword filtered: '

(706, 20)

In [9]:
df_jdg_ = keywords_filtering(df_jdg, keywords_1969)
df_jdg_ = unwanted_filtering(df_jdg_, unwanted)

===KEYWORDS===


'After keyword filtered: '

(863, 20)

=====UNWANTED====


'After keyword filtered: '

(863, 20)

* Export iramutek

In [10]:
creating_iramutek_file(df_all_, "../data/iramutek/corpus_all" + date +".txt")
creating_iramutek_file(df_jdg_, "../data/iramutek/corpus_jdg" + date +".txt")
creating_iramutek_file(df_gdl_, "../data/iramutek/corpus_gdl" + date +".txt")